In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sn 
from seaborn import distplot
from seaborn import heatmap
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import math
import time
import numpy as np
from tqdm import tqdm
import multiprocessing
import pickle
from math import isinf 
from scipy.interpolate import interp1d
from numpy import array, zeros, full, argmin, inf, ndim
from sklearn import preprocessing


import statistics
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier as RB
#from sklearn.decomposition import KernelPCA
#from imblearn.over_sampling import SMOTE as SM
from sklearn.ensemble import RandomForestClassifier as RF,GradientBoostingRegressor as GB,ExtraTreesClassifier as ET
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neural_network import MLPClassifier as MP

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
 
# Scikit-Learn imports
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [4]:
!pip install -q shap

     |████████████████████████████████| 356 kB 6.7 MB/s 


In [5]:
!pip install -q eli5

     |████████████████████████████████| 106 kB 8.0 MB/s 


In [6]:
import shap
import eli5
from eli5.sklearn import PermutationImportance
import xgboost as xgb

from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.utils.class_weight import compute_class_weight
from sklearn.inspection import plot_partial_dependence

from sklearn.metrics import accuracy_score, precision_recall_curve, roc_auc_score, roc_curve, f1_score, auc

The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.


In [8]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [0.1, 0.2, 0.25, 0.3, 1],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 500, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)

In [ ]:
#grid_search.fit(X_train_ctgan, y_train_ctgan)
#grid_search.best_params_
#best_grid = grid_search.best_estimator_

In [9]:
estimator = xgb.XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    random_state = 1)

In [10]:
parameters = {
    'max_depth': [0.2, 0.4, 0.6, 0.8, 1],
    'n_estimators': [100, 200, 400, 500, 800],
    'learning_rate': [0.1, 0.01, 0.03, 0.07, 0.1]
}

In [11]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [ ]:
#grid_search.fit(X_train_ctgan, y_train_ctgan)

In [ ]:
grid_search.best_estimator_

In [ ]:
rnd_fr_clf = RandomForestClassifier(n_estimators=500,
                                max_features=0.25,
                                criterion="entropy",
                                class_weight="balanced",
                                random_state=1)

scores = cross_val_score(rnd_fr_clf,
                         X_train_ctgan, 
                         y_train_ctgan,
                         scoring="roc_auc", 
                         cv=10, 
                         n_jobs=-1, 
                         verbose=50)
print("Baseline model (RandomForest) Mean AUC:", scores.mean())

In [ ]:
#train_acc = rnd_fr_clf.score(latent_vector_train, y_train)
# y_pred = rnd_fr_clf.predict(x_test)
# test_acc = accuracy_score(y_test, y_pred)
# print(train_acc,test_acc)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

rnd_fr_clf.fit(latent_vector_train, y_train)
# predict probabilities
probs = rnd_fr_clf.predict_proba(latent_vector)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# predict class values
# y_pred = rnd_fr_clf.predict(x_test)

# plot the ROC curve
rnd_fr_fpr, rnd_fr_tpr, rnd_fr_th = roc_curve(y_test, probs)
ax[0].plot(rnd_fr_fpr, rnd_fr_tpr, label='Baseline Random Forest')
ax[0].set_xlabel("False Positive")
ax[0].set_ylabel("True Positive")
ax[0].legend()
ax[0].set_title("ROC curve")
# plot the precision-recall curve
rnd_fr_precision, rnd_fr_recall, _ = precision_recall_curve(y_test, probs)
ax[1].plot(rnd_fr_recall, rnd_fr_precision, label='Baseline Random Forest')
ax[1].set_xlabel('Recall')
ax[1].set_ylabel('Precision')
ax[1].legend()
ax[1].set_title("Precision Recall curve")
plt.show()

In [ ]:
# Define base learners
rnd_fr_clf = RandomForestClassifier(n_estimators=300,
                                max_features="sqrt",
                                criterion="gini",
                                min_samples_leaf=5,
                                class_weight="balanced",
                                random_state=1)

xgb_clf = xgb.XGBClassifier(objective="binary:logistic",
                            learning_rate=0.03,
                            n_estimators=500,
                            max_depth=1,
                            subsample=0.4,
                            random_state=1)

# Define meta-learner
logreg_clf = LogisticRegression(penalty="l2",
                                C=100,
                                fit_intercept=True)

# average ensemble
voting_clf = VotingClassifier([("xgb", xgb_clf),
                               ("rf", rnd_fr_clf)],
                              voting="soft",
                              flatten_transform=True)
voting_clf.fit(latent_vector_train, y_train)
xgb_model, rf_model = voting_clf.estimators_

In [ ]:
models = {"xgb": xgb_model, "rf": rf_model, "avg_ensemble": voting_clf}

In [ ]:
# Build first stack of base learners
average_ensemble = make_pipeline(voting_clf,
                            FunctionTransformer(lambda X: X[:, 1::2]))
# Use CV to generate meta-features
meta_features = cross_val_predict(average_ensemble,
                                  X_train_ctgan,y_train_ctgan,
                                  cv=10,
                                  method="transform",
                                  n_jobs=-1,
                                  verbose=50)

In [ ]:
# Refit the first stack on the full training set
average_ensemble.fit(X_train_ctgan,y_train_ctgan)
# Fit the meta learner
stacked_ensemble = logreg_clf.fit(meta_features, y_train)

In [ ]:
# Plot ROC and PR curves using all models and test data
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
for name, model in models.items():
    model_probs = model.predict_proba(latent_vector)[:, 1:]
    model_auc_score = roc_auc_score(y_test, model_probs)
    fpr, tpr, th = roc_curve(y_test, model_probs)
    precision, recall, _ = precision_recall_curve(y_test, model_probs)
    ax[0].plot(fpr, tpr, label=name + str(model_auc_score))
    ax[1].plot(recall, precision, label=name)

stacked_probs = stacked_ensemble.predict_proba(average_ensemble.transform(latent_vector))[:, 1:]
stacked_auc_score = roc_auc_score(y_test, stacked_probs)
fpr, tpr, th = roc_curve(y_test, stacked_probs)
ax[0].plot(fpr, tpr, label="stacked_ensemble"+ str(stacked_auc_score))
ax[0].set_xlabel("False Positive")
ax[0].set_ylabel("True Positive")
ax[0].legend()
ax[0].set_title("ROC curve")
ax[1].set_xlabel('Recall')
ax[1].set_ylabel('Precision')
ax[1].legend()
ax[1].set_title("Precision Recall curve")
plt.show()

In [ ]:
y_prob= stacked_ensemble.predict_proba(average_ensemble.transform(latent_vector))[:, 1:]
y_pred= stacked_ensemble.predict(average_ensemble.transform(latent_vector))
print('Confusion Matrix')
print('='*60)
print(confusion_matrix(y_test,y_pred),"\n")
print('Classification Report')
print('='*60)
print(classification_report(y_test,y_pred),"\n")
print('AUC-ROC')
print('='*60)
print(roc_auc_score(y_test, y_prob))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))
print(f1_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_pred)

In [ ]:
from sklearn.metrics import auc, plot_precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# Use AUC function to calculate the area under the curve of precision recall curve
auc_precision_recall = auc(recall, precision)
print(auc_precision_recall)